In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms,models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import kagglehub
from pathlib import Path
import os
from torch.utils.data import random_split
import torch.nn.functional as F
from PIL import Image
from PIL import UnidentifiedImageError
import neptune


In [27]:
path = kagglehub.dataset_download("shaunthesheep/microsoft-catsvsdogs-dataset")
print(path)
data_dir = Path(path) / "PetImages"
print(data_dir)

/kaggle/input/microsoft-catsvsdogs-dataset
/kaggle/input/microsoft-catsvsdogs-dataset/PetImages


In [28]:
class SafeImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        try:
            return super().__getitem__(index)
        except (OSError, UnidentifiedImageError, ValueError) as e:
            print(f"Пропущен битый файл: {self.samples[index][0]} ({e})")
            
            new_index = (index + 1) % len(self.samples)
            return self.__getitem__(new_index)

In [29]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

dataset=SafeImageFolder(root=data_dir,transform=transform)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])



train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True,num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False,num_workers=4)
num_classes=2


In [30]:
class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(3,6,kernel_size=5,stride=1,padding=2)
        self.pool1=nn.AvgPool2d(2,2)
        self.conv2=nn.Conv2d(6,16,kernel_size=5)
        self.pool2=nn.AvgPool2d(2,2)
        self.conv3=nn.Conv2d(16,120,kernel_size=5)
        
        self.fc1 = None  
        self.fc2 = nn.Linear(84, num_classes)


    def forward_0(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        return x 

    def forward(self,x):
        x=self.forward_0(x)
        if self.fc1 is None:
            flattened_dim = x.shape[1] * x.shape[2] * x.shape[3]
            self.fc1 = nn.Linear(flattened_dim, 84).to(x.device)
        
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [31]:
def evaluate(model,dataloader,device):
    model.eval()
    correct=0
    total=0
    running_loss=0.0
    with torch.no_grad():
        for images,labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs=model(images)
            loss=criterion(outputs,labels)
            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    avg_loss = running_loss / total
    accuracy = correct / total
    model.train()
    return avg_loss, accuracy

In [32]:

run = neptune.init_run(
    project="kulikovka/AlexLenNets",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJiNjZjNWE3OS1jMzRlLTQyZjAtYmFiNi04ZGU1ZjY0MDAxNzgifQ=="
)


device = torch.device("cuda")
model=LeNet5().to(device)
criterion=nn.CrossEntropyLoss()
optimazer=optim.Adam(model.parameters(),lr=0.001)
num_epochs=15

train_losses=[]
train_accs=[]
test_losses=[]
test_accs=[]

run['parameters'] = {
    "epochs": num_epochs,
    "device": str(device),
    "train_size": len(train_dataset),
    "test_size": len(test_dataset)
}



[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/kulikovka/AlexLenNets/e/AL-10


In [33]:

for epoch in range(1, num_epochs + 1):
    model.train()
    epoch_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")

    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)
        optimazer.zero_grad()           
        outputs = model(images)         
        loss = criterion(outputs, labels)  
        loss.backward()                 
        optimazer.step()                

        epoch_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        pbar.set_postfix({"batch_loss": loss.item(), "acc": f"{100 * correct / total}%"})

    train_loss = epoch_loss / len(train_dataset)
    train_acc = correct / total
    test_loss, test_acc = evaluate(model, test_loader,device)

    print(f"Epoch {epoch}: Train loss={train_loss}, Train acc={train_acc}, "
          f"Test loss={test_loss}, Test acc={test_acc}")
    run["train/loss"].append(train_loss)
    run["train/accuracy"].append(train_acc)
    run["test/loss"].append(test_loss)
    run["test/accuracy"].append(test_acc)
    run["epoch"].append(epoch)
run.stop()

Epoch 1/15:   0%|          | 1/313 [00:01<09:25,  1.81s/it, batch_loss=0.692, acc=53.125%]

Пропущен битый файл: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg (cannot identify image file <_io.BufferedReader name='/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg'>)


Epoch 1/15: 100%|██████████| 313/313 [00:27<00:00, 11.48it/s, batch_loss=0.672, acc=56.57%]             
/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Пропущен битый файл: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat/666.jpg (cannot identify image file <_io.BufferedReader name='/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat/666.jpg'>)
Epoch 1: Train loss=0.6779731781005859, Train acc=0.5657, Test loss=0.652942924118042, Test acc=0.6178


Epoch 2/15:  62%|██████▏   | 193/313 [00:17<00:09, 12.48it/s, batch_loss=0.636, acc=62.28950777202073%] 

...

Пропущен битый файл: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat/666.jpg (cannot identify image file <_io.BufferedReader name='/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat/666.jpg'>)
Epoch 14: Train loss=0.45385634846687317, Train acc=0.78865, Test loss=0.5287555519580841, Test acc=0.7382


Epoch 15/15:  51%|█████▏    | 161/313 [00:15<00:12, 11.71it/s, batch_loss=0.379, acc=79.7360248447205%] 

Пропущен битый файл: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg (cannot identify image file <_io.BufferedReader name='/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg'>)


Epoch 15/15: 100%|██████████| 313/313 [00:27<00:00, 11.35it/s, batch_loss=0.45, acc=79.505%]            
/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Пропущен битый файл: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat/666.jpg (cannot identify image file <_io.BufferedReader name='/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat/666.jpg'>)
Epoch 15: Train loss=0.4429778965950012, Train acc=0.79505, Test loss=0.5385178909301758, Test acc=0.7368
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 5 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 5 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/kulikovka/AlexLenNets/e/AL-10/metadata


In [79]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 4096),  
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = torch.flatten(x, 1)
        x = self.fc_layers(x)
        return x

In [80]:
run = neptune.init_run(
    project="kulikovka/AlexLenNets",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJiNjZjNWE3OS1jMzRlLTQyZjAtYmFiNi04ZGU1ZjY0MDAxNzgifQ=="
)


# ---------------- Training ----------------
model = AlexNet(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
num_epochs = 15


run['parameters'] = {
    "epochs": num_epochs,
    "device": str(device),
    "train_size": len(train_dataset),
    "test_size": len(test_dataset),
    "lr": 0.01
}


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/kulikovka/AlexLenNets/e/AL-24


In [81]:

# ---------------- Neptune ----------------

for epoch in range(1, num_epochs + 1):
    model.train()
    epoch_loss, correct, total = 0.0, 0, 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")
    
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        
        pbar.set_postfix({"batch_loss": loss.item(), "acc": f"{100*correct/total:.2f}%"})
    
    train_loss = epoch_loss / len(train_dataset)
    train_acc = correct / total
    test_loss, test_acc = evaluate(model, test_loader, device)
    
    print(f"Epoch {epoch}: Train loss={train_loss:.4f}, Train acc={train_acc:.4f}, "
          f"Test loss={test_loss:.4f}, Test acc={test_acc:.4f}")
    
    run["train/loss"].append(train_loss)
    run["train/accuracy"].append(train_acc)
    run["test/loss"].append(test_loss)
    run["test/accuracy"].append(test_acc)
    run["epoch"].append(epoch)

run.stop()


Epoch 1/15:  19%|█▉        | 61/313 [00:12<00:37,  6.65it/s, batch_loss=0.693, acc=51.31%]

Пропущен битый файл: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat/666.jpg (cannot identify image file <_io.BufferedReader name='/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat/666.jpg'>)


Epoch 1/15:  46%|████▌     | 143/313 [00:23<00:23,  7.37it/s, batch_loss=0.695, acc=52.90%]/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Epoch 1/15:  78%|███████▊  | 243/313 [00:38<00:09,  7.37it/s, batch_loss=0.681, acc=55.06%]

Пропущен битый файл: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg (cannot identify image file <_io.BufferedReader name='/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg'>)


Epoch 1/15: 100%|██████████| 313/313 [00:48<00:00,  6.44it/s, batch_loss=0.593, acc=55.88%]


...

Пропущен битый файл: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg (cannot identify image file <_io.BufferedReader name='/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg'>)


Epoch 15/15:  54%|█████▍    | 170/313 [00:27<00:20,  7.04it/s, batch_loss=0.156, acc=93.05%] /usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Epoch 15/15:  66%|██████▌   | 207/313 [00:32<00:13,  7.61it/s, batch_loss=0.101, acc=93.06%] 

Пропущен битый файл: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat/666.jpg (cannot identify image file <_io.BufferedReader name='/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat/666.jpg'>)


Epoch 15/15: 100%|██████████| 313/313 [00:47<00:00,  6.59it/s, batch_loss=0.171, acc=93.00%] 


Epoch 15: Train loss=0.1727, Train acc=0.9300, Test loss=0.2177, Test acc=0.9062
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 5 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 5 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/kulikovka/AlexLenNets/e/AL-24/metadata


In [82]:
alexnet = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)



In [83]:
for param in alexnet.features.parameters():
    param.requires_grad = False

In [84]:
alexnet.classifier = nn.Sequential(
    nn.Linear(256 * 6 * 6, 4096),
    nn.ReLU(inplace=True),
    nn.Linear(4096, 1024),
    nn.ReLU(inplace=True),
    nn.Linear(1024, num_classes)
)



In [87]:
run = neptune.init_run(
    project="kulikovka/AlexLenNets",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJiNjZjNWE3OS1jMzRlLTQyZjAtYmFiNi04ZGU1ZjY0MDAxNzgifQ=="
)

model = alexnet.to(device)
trainable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.Adam(trainable_params, lr=0.001)
criterion=nn.CrossEntropyLoss()

num_epochs=5

train_losses=[]
train_accs=[]
test_losses=[]
test_accs=[]

run['parameters'] = {
    "epochs": num_epochs,
    "device": str(device),
    "train_size": len(train_dataset),
    "test_size": len(test_dataset)
}

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/kulikovka/AlexLenNets/e/AL-26


In [88]:
for epoch in range(1, num_epochs + 1):
    model.train()
    epoch_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")

    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()           
        outputs = model(images)         
        loss = criterion(outputs, labels)  
        loss.backward()                 
        optimizer.step()                

        epoch_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        pbar.set_postfix({"batch_loss": loss.item(), "acc": f"{100 * correct / total}%"})

    train_loss = epoch_loss / len(train_dataset)
    train_acc = correct / total
    test_loss, test_acc = evaluate(model, test_loader,device)

    print(f"Epoch {epoch}: Train loss={train_loss}, Train acc={train_acc}, "
          f"Test loss={test_loss}, Test acc={test_acc}")
    run["train/loss"].append(train_loss)
    run["train/accuracy"].append(train_acc)
    run["test/loss"].append(test_loss)
    run["test/accuracy"].append(test_acc)
    run["epoch"].append(epoch)

run.stop()

Epoch 1/5:   4%|▎         | 11/313 [00:05<01:09,  4.35it/s, batch_loss=0.101, acc=95.83333333333333%]/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Epoch 1/5:  27%|██▋       | 84/313 [00:14<00:28,  8.15it/s, batch_loss=0.0707, acc=95.72172619047619%]

Пропущен битый файл: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg (cannot identify image file <_io.BufferedReader name='/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg'>)


Epoch 1/5:  35%|███▍      | 108/313 [00:18<00:24,  8.22it/s, batch_loss=0.166, acc=95.52951388888889%] 

Пропущен битый файл: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat/666.jpg (cannot identify image file <_io.BufferedReader name='/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat/666.jpg'>)


Epoch 1/5: 100%|██████████| 313/313 [00:45<00:00,  6.84it/s, batch_loss=0.133, acc=95.405%]            


...

Epoch 5/5:   1%|          | 3/313 [00:04<05:28,  1.06s/it, batch_loss=0.0109, acc=98.4375%]

Epoch 5/5:  65%|██████▍   | 203/313 [00:30<00:13,  8.16it/s, batch_loss=0.0956, acc=97.38051470588235%] /usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Epoch 5/5:  66%|██████▌   | 207/313 [00:31<00:12,  8.41it/s, batch_loss=0.0183, acc=97.36328125%]      

Пропущен битый файл: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg (cannot identify image file <_io.BufferedReader name='/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg'>)


Epoch 5/5: 100%|██████████| 313/313 [00:45<00:00,  6.89it/s, batch_loss=0.121, acc=97.23%]              


Epoch 5: Train loss=0.0740445006147027, Train acc=0.9723, Test loss=0.13322922866344453, Test acc=0.9498
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 5 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 5 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/kulikovka/AlexLenNets/e/AL-26/metadata


**LeNet5:** train acc-**0.79505** vs test acc-**0.7368** train loss-**0.443** vs test loss-**0.54*

**AlexNet5:** train acc-**0.9300** vs test acc-**0.9062** train loss-**0.17** vs test loss-**0.22*

**AlexNet5_stilled:** train acc-**0.9723** vs test acc-**0.9498** train loss-**0.07** vs test loss-**0.13*

### ***AlexNet5_stilled*** показала лучшие метрики и понадобилось всего **5 эпох** vs **15** у ***LeNet5*** и ***AlexNet5***.